# FCC Application Scraping

## Set up selenium webdriver with Firefox
* Download Firefox driver [geckodriver](https://github.com/mozilla/geckodriver/releases) for selenium
* Move the driver executable into the same directory as this notebook (or update the path to the executable in `scrape_table_for_call_sign`)
* `pip install selenium`

In [1]:
from bs4 import BeautifulSoup
import time
import random
from math import sqrt
import csv

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options  
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException        

In [3]:
import os
import pandas as pd

## Define functions for scraping table for an individual call sign

In [4]:
def element_exists(driver, xpath):
    """
    Helper function that checks if the element at the given xpath exists on
    driver's current page. This is used to check if there is a "Next" button
    -- to scrape through multiple pages of the FCC's paginated tables.
    """
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True


In [5]:
def extract_callsign(innerHTML):
    '''
    Helper function that extracts the callsign from Application search results page.
    This is used to append the CALLSIGN to each row when scraping table data
    '''
    soup = BeautifulSoup(innerHTML, "lxml")
    
    # tag attributes for element containing text "Call Sign = " 
    tag_attrs = {"class":"cell-pri-light", "valign":"bottom", "align":"left"}
    
    # extract callsign as text
    callsign = (
        soup.find_all(attrs=tag_attrs, limit=1)[0]
        .find_all("b")[0]
        .get_text()
    )
    
    return callsign

In [6]:
def count_results(innerHTML):
    '''
    Helper function that extracts the total number of results across all pages.
    This is used to check the scraped table data has the expected number of rows.
    '''
    
    soup = BeautifulSoup(innerHTML, "lxml")
    
    # tag attributes for element containing text "Matches 1-47 (of 47)"
    tag_attrs = {"class":"cell-pri-dark", "width":"35%", "valign":"middle", "align":"left"}

    # extract results count from inside "(of 47)" as integer 47
    count = int(soup.find_all(attrs=tag_attrs, limit=1)[0]
                .find_all("b")[1]
                .get_text()
               )
    
    return count

In [7]:
def scrape_table_page(innerHTML):
    """
    Extracts the following details from a call sign table page:
        * File Number
        * Call Sign/Lease ID
        * Applicant Name
        * FRN
        * Purpose - split into Purpose_Main and Purpose_Note
        * Radio
        * Service
        * Receipt
        * Date
        * Status
    Returns as a list of lists.
    """
    
    # Extract search data from innerHTML soup
    soup = BeautifulSoup(innerHTML, "lxml")

    CALLSIGN = extract_callsign(innerHTML)
    table_page = soup.find("table", {"summary": "Application search results"})
    
    # Extract search results as list of rows
    rows = table_page.find_all("tr")[1:-1] # note: skip header and pagination rows
    
    # Extract results by row into table, attaching callsign number
    table_data = []
    
    for row in rows:  
        
        row_data = [CALLSIGN, ]
        cols = row.find_all("td")
        
        # extract values from each cell in row
        for i in range(1, len(cols)): # note: skip the first (index) cell
            
            # extract cell data, trim leading/trailing whitespace
            cell_data = cols[i].get_text().strip()
            
            # fifth-index cells "Purpose" has to get split into two cols
            if i == 5:
                
                purpose = cell_data.split("\n")
                purpose_main = purpose[0].strip()
                purpose_note = ""
                
                # sometimes the "Purpose" column has an optional note
                if len(purpose) >= 2:
                    purpose_note = purpose[-1].strip()
                    
                row_data.append(purpose_main)
                row_data.append(purpose_note)
            
            # other cells can be inserted as normal
            else:
                row_data.append(cell_data)
                
        table_data.append(row_data)
    
    return(table_data)


In [8]:
def cache_to_csv(lst, path):

    no_rows = len(lst)
    
    with open(path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(lst)

    with open(path, 'r', newline='') as f:
        reader = csv.reader(f)
        check = list(reader)

    assert(len(lst) == len(check))
    
    print('LOG[INFO]: Wrote %s rows to %s' % (no_rows, path))

In [9]:
def make_html_path(parent_dir, call_sign, page_no):
    '''
    Helper function that generates path for caching html pages
    '''
    path = ("%s/%s-%s.html" % (parent_dir, call_sign, str(page_no).zfill(3)))
    return path

In [10]:
def scrape_tables_for_call_sign(call_sign, html_cache, csv_cache, without_browser=False):
    """
    Scrapes the FCC Application Search database for a given call sign.
    
    Caches results pages as HTML
    
    Returns the table as a list of lists, for example:
    [
      ['File Number', 'Call Sign/Lease ID', 'Applicant Name', 'FRN', 'Purpose_Main', 'Purpose_Note', 'Radio Service', 'Receipt Date', 'Status'],
      ['0000214527',  'KNKN555',            'AT&T',           '',    'Amendment',    'Transfer Control', 'AL',        '09/05/2000',   'Granted'],
      ...,
      ...
    ]
    
    """
    tick = time.time()
    print("LOG[INFO]: START %s " % call_sign)
    
    
    # driver config - start in private
    firefox_profile = webdriver.FirefoxProfile()
    firefox_profile.set_preference("browser.privatebrowsing.autostart", True)
    
    # (optional) don't open up browser
    firefox_options = Options()
    if without_browser:
        firefox_options.add_argument("--headless")
    
    # spin up the webdriver
    firefox = webdriver.Firefox(options=firefox_options, executable_path="./geckodriver")
    firefox.get("https://wireless2.fcc.gov/UlsApp/ApplicationSearch/searchAdvanced.jsp")
    
    # specify the call sign that we want to scrape for
    callsign = firefox.find_element_by_id("ulsCallSign")
    callsign.send_keys(call_sign)
    
    # set pagination to 100 (maximum)
    pagination = Select(firefox.find_element_by_xpath("//select[@name='pageSize']"))
    pagination.select_by_visible_text("100")
    
    # submit the search form, wait for load
    # NOTE: find form submission by form element name, not search button image
    firefox.find_element_by_name("search").submit()
    WebDriverWait(firefox, 45).until(EC.url_changes("https://wireless2.fcc.gov/UlsApp/ApplicationSearch/searchAdvanced.jsp"))
    WebDriverWait(firefox, 45).until(lambda _: firefox.execute_script("return document.readyState") == "complete")
    
    # extract and cache html from first page
    page = 1
    innerHTML = firefox.execute_script("return document.body.innerHTML")
    with open(make_html_path(html_cache, call_sign, page), 'w') as f:
        f.write(innerHTML)
    
    # store count of search results for later check
    no_results = count_results(innerHTML)
    
    # Scrape results data from first page
    search_data = []
    search_data += scrape_table_page(innerHTML)
    
    # as long there is a link to "next page"...
    next_page_xpath = "//*[@title='Next page of results']"
    while element_exists(firefox, next_page_xpath):
        # ... go to next page
        firefox.find_element_by_xpath(next_page_xpath).click()
        # ... wait for load
        WebDriverWait(firefox, 45).until(lambda _: firefox.execute_script("return document.readyState") == "complete")
        # .. increment page counter, extract and cache html 
        page += 1
        innerHTML = firefox.execute_script("return document.body.innerHTML")
        with open(make_html_path(html_cache, call_sign, page), 'w') as f:
            f.write(innerHTML)
        # ... scrape and add page results to all results
        search_data += scrape_table_page(innerHTML)

    # gracefully close the webdriver
    firefox.quit()
    
    # Check that all results have been scraped
    no_rows = len(search_data)
    #assert(no_rows == no_results)
    print("LOG[INFO]: Scraped %s rows out of %s results" % (no_rows, no_results))

    # Cache search results table to csv (all results from all pages)
    csv_path = ''.join([csv_cache, '/', call_sign, ".csv"])
    cache_to_csv(search_data, csv_path)
    

    tock = time.time()
    print("LOG[INFO]: END %s (%2f seconds)" % (call_sign, tock - tick))
    
    return search_data
     

## Scrape the tables for a list of call signs (loop)

In [11]:
# read in list of all callsigns from file

callsign_path = 'data-cache/cma_callsign_lookup.csv'
callsigns = pd.read_csv(callsign_path).sort_values(by=['callsign_leaseID'])['callsign_leaseID'].tolist()

In [12]:
# Set paths for caching

html_cache = 'search-cache/html'
csv_cache = 'search-cache/csv'

## LOG Attempts

### Attempt 1
LOG[INFO]: START TIME is 04:35 PM
LOG[INFO]: START B 
LOG[INFO]: Scraped 8582 rows out of 8582 results
LOG[INFO]: Wrote 8582 rows to search-cache/csv/B.csv
LOG[INFO]: END B (377.532821 seconds)

### Attempt 2
start_index = 1, 2, 3
debugging runs
Scraped KNKA200, 201, 202

### Attempt 3
EVERYTHING til 1000
start_index = KNKA203 [4]
timeout at KNKA636 [434]

### ATTEMPT 4
```
LOG[INFO]: START TIME is 02:52 PM
LOG[INFO]: END TIME is 07:47 PM (TOTAL: 295.3278821190198 minutes)
```

Completed searches from KNKA636 [434] to KNKN585 [999].

### ATTEMPT 5

```
LOG[INFO]: START TIME is 02:15 PM
LOG[INFO]: END TIME is 11:14 AM (TOTAL: 1259.371838748455 minutes)
```
Completed searches from KNKN586 [1000] to WRBT612 [2602].

Errors at: [1470, 2022, 2300, 2442]
```
Error at KNKQ233, located at callsigns[1470]
Error: Message: Socket timeout reading Marionette handshake data: Resource temporarily unavailable (os error 35)

Error at L000004840, located at callsigns[2022]
Error: Message: 

Error at M, located at callsigns[2300]
Error: Message: 

Error at WPQS659, located at callsigns[2442]
Error: list index out of range
```

Missing: [2603] = ['WRBZ793']


In [36]:
print('Attempt 1\n\n',In[15])

Attempt 1

 tick = time.time()
print("LOG[INFO]: START TIME is %s\n" % (time.strftime("%I:%M %p")))

tries = 0
max_tries = 5

start_index = 2 # skip call_sign = 'B'

error_locs = []

for cs in callsigns[start_index:3]:    
    if tries < max_tries:
        try:
            scrape_tables_for_call_sign(cs, html_cache, csv_cache, without_browser=False)
            pause = (random.randint(1,30))
        except Exception as e:
            tries += 1
            error_index = callsigns.index(cs)
            error_locs.append(error_index)
            print("Error at %s, located at callsigns[%s]" % (cs, error_index))
            print("Error:", e)
            pause = (random.randint(200,300) * sqrt(tries))
        finally:
            time.sleep(pause)
    else:
        print("LOG[INFO]: Maximum number of errors (%s) reached." % (tries))
        print("Errors at:", error_locs)
        break

tock = time.time()
print("LOG[INFO]: END TIME is %s (TOTAL: %s minutes)" 
      % (time.strftime("%I:%M

In [35]:
print('Attempt 2\n\n',In[16])

Attempt 2

 tick = time.time()
print("LOG[INFO]: START TIME is %s\n" % (time.strftime("%I:%M %p")))

tries = 0
max_tries = 5

start_index = 3 # skip call_sign = 'B'

error_locs = []
error_index = 0

for cs in callsigns[start_index:4]:    
    if tries < max_tries:
        try:
            scrape_tables_for_call_sign(cs, html_cache, csv_cache, without_browser=False)
            pause = (random.randint(1,30))
            time.sleep(pause)
        except Exception as e:
            tries += 1
            error_index = callsigns.index(cs)
            error_locs.append(error_index)
            print("Error at %s, located at callsigns[%s]" % (cs, error_index))
            print("Error:", e)
            pause = (random.randint(200,300) * sqrt(tries))
            time.sleep(pause)
        finally:
            pass
    else:
        print("LOG[INFO]: Maximum number of errors (%s) reached." % (tries))
        print("Errors at:", error_locs)
        break

tock = time.time()
print("LOG[INFO]: END

In [34]:
print('Attempt 3\n\n',In[17])

Attempt 3

 tick = time.time()
print("LOG[INFO]: START TIME is %s\n" % (time.strftime("%I:%M %p")))

tries = 0
max_tries = 5

start_index = 4 # see attempts log below

error_locs = []
error_index = start_index

for cs in callsigns[start_index:1000]:    
    if tries < max_tries:
        try:
            scrape_tables_for_call_sign(cs, html_cache, csv_cache, without_browser=True)
            pause = (random.randint(1,30))
            time.sleep(pause)
        except Exception as e:
            tries += 1
            error_index = callsigns.index(cs)
            error_locs.append(error_index)
            print("Error at %s, located at callsigns[%s]" % (cs, error_index))
            print("Error:", e)
            pause = (random.randint(200,300) * sqrt(tries))
            time.sleep(pause)
        finally:
            pass
    else:
        print("LOG[INFO]: Maximum number of errors (%s) reached." % (tries))
        print("Errors at:", error_locs)
        break

tock = time.time()
print("

In [18]:
## attempt 4 (In[18])

tick = time.time()
print("LOG[INFO]: START TIME is %s\n" % (time.strftime("%I:%M %p")))

tries = 0
max_tries = 5

start_index = 434 # see attempts log below

error_locs = []
error_index = start_index

for cs in callsigns[start_index:1000]:    
    if tries < max_tries:
        try:
            scrape_tables_for_call_sign(cs, html_cache, csv_cache, without_browser=True)
            pause = (random.randint(1,30))
            time.sleep(pause)
        except Exception as e:
            tries += 1
            error_index = callsigns.index(cs)
            error_locs.append(error_index)
            print("Error at %s, located at callsigns[%s]" % (cs, error_index))
            print("Error:", e)
            pause = (random.randint(500,800) * sqrt(tries))
            time.sleep(pause)
        finally:
            pass
    else:
        print("LOG[INFO]: Maximum number of errors (%s) reached." % (tries))
        print("Errors at:", error_locs)
        break

tock = time.time()
print("LOG[INFO]: END TIME is %s (TOTAL: %s minutes)" 
      % (time.strftime("%I:%M %p"), ((tock - tick)/60))
     )
print("Completed searches from %s [%s] to %s [%s]." 
      % (callsigns[start_index], start_index, cs, error_index)
     )

LOG[INFO]: START TIME is 02:52 PM

LOG[INFO]: START KNKA636 
LOG[INFO]: Scraped 45 rows out of 45 results
LOG[INFO]: Wrote 45 rows to search-cache/csv/KNKA636.csv
LOG[INFO]: END KNKA636 (16.271517 seconds)
LOG[INFO]: START KNKA637 
LOG[INFO]: Scraped 59 rows out of 59 results
LOG[INFO]: Wrote 59 rows to search-cache/csv/KNKA637.csv
LOG[INFO]: END KNKA637 (15.853078 seconds)
LOG[INFO]: START KNKA638 
LOG[INFO]: Scraped 79 rows out of 79 results
LOG[INFO]: Wrote 79 rows to search-cache/csv/KNKA638.csv
LOG[INFO]: END KNKA638 (14.937548 seconds)
LOG[INFO]: START KNKA639 
LOG[INFO]: Scraped 52 rows out of 52 results
LOG[INFO]: Wrote 52 rows to search-cache/csv/KNKA639.csv
LOG[INFO]: END KNKA639 (15.301425 seconds)
LOG[INFO]: START KNKA640 
LOG[INFO]: Scraped 44 rows out of 44 results
LOG[INFO]: Wrote 44 rows to search-cache/csv/KNKA640.csv
LOG[INFO]: END KNKA640 (14.515842 seconds)
LOG[INFO]: START KNKA641 
LOG[INFO]: Scraped 34 rows out of 34 results
LOG[INFO]: Wrote 34 rows to search-cach

LOG[INFO]: START KNKA684 
LOG[INFO]: Scraped 36 rows out of 36 results
LOG[INFO]: Wrote 36 rows to search-cache/csv/KNKA684.csv
LOG[INFO]: END KNKA684 (13.561410 seconds)
LOG[INFO]: START KNKA685 
LOG[INFO]: Scraped 82 rows out of 82 results
LOG[INFO]: Wrote 82 rows to search-cache/csv/KNKA685.csv
LOG[INFO]: END KNKA685 (12.194613 seconds)
LOG[INFO]: START KNKA686 
LOG[INFO]: Scraped 63 rows out of 63 results
LOG[INFO]: Wrote 63 rows to search-cache/csv/KNKA686.csv
LOG[INFO]: END KNKA686 (14.631197 seconds)
LOG[INFO]: START KNKA687 
LOG[INFO]: Scraped 75 rows out of 75 results
LOG[INFO]: Wrote 75 rows to search-cache/csv/KNKA687.csv
LOG[INFO]: END KNKA687 (13.108890 seconds)
LOG[INFO]: START KNKA688 
LOG[INFO]: Scraped 53 rows out of 53 results
LOG[INFO]: Wrote 53 rows to search-cache/csv/KNKA688.csv
LOG[INFO]: END KNKA688 (12.857307 seconds)
LOG[INFO]: START KNKA689 
LOG[INFO]: Scraped 41 rows out of 41 results
LOG[INFO]: Wrote 41 rows to search-cache/csv/KNKA689.csv
LOG[INFO]: END KN

LOG[INFO]: START KNKA733 
LOG[INFO]: Scraped 49 rows out of 49 results
LOG[INFO]: Wrote 49 rows to search-cache/csv/KNKA733.csv
LOG[INFO]: END KNKA733 (13.699061 seconds)
LOG[INFO]: START KNKA734 
LOG[INFO]: Scraped 44 rows out of 44 results
LOG[INFO]: Wrote 44 rows to search-cache/csv/KNKA734.csv
LOG[INFO]: END KNKA734 (15.259693 seconds)
LOG[INFO]: START KNKA735 
LOG[INFO]: Scraped 64 rows out of 64 results
LOG[INFO]: Wrote 64 rows to search-cache/csv/KNKA735.csv
LOG[INFO]: END KNKA735 (13.179298 seconds)
LOG[INFO]: START KNKA736 
LOG[INFO]: Scraped 48 rows out of 48 results
LOG[INFO]: Wrote 48 rows to search-cache/csv/KNKA736.csv
LOG[INFO]: END KNKA736 (11.455101 seconds)
LOG[INFO]: START KNKA737 
LOG[INFO]: Scraped 67 rows out of 67 results
LOG[INFO]: Wrote 67 rows to search-cache/csv/KNKA737.csv
LOG[INFO]: END KNKA737 (13.189288 seconds)
LOG[INFO]: START KNKA738 
LOG[INFO]: Scraped 74 rows out of 74 results
LOG[INFO]: Wrote 74 rows to search-cache/csv/KNKA738.csv
LOG[INFO]: END KN

LOG[INFO]: START KNKA783 
LOG[INFO]: Scraped 56 rows out of 56 results
LOG[INFO]: Wrote 56 rows to search-cache/csv/KNKA783.csv
LOG[INFO]: END KNKA783 (13.789274 seconds)
LOG[INFO]: START KNKA784 
LOG[INFO]: Scraped 51 rows out of 51 results
LOG[INFO]: Wrote 51 rows to search-cache/csv/KNKA784.csv
LOG[INFO]: END KNKA784 (13.331619 seconds)
LOG[INFO]: START KNKA785 
LOG[INFO]: Scraped 128 rows out of 128 results
LOG[INFO]: Wrote 128 rows to search-cache/csv/KNKA785.csv
LOG[INFO]: END KNKA785 (16.071637 seconds)
LOG[INFO]: START KNKA786 
LOG[INFO]: Scraped 36 rows out of 36 results
LOG[INFO]: Wrote 36 rows to search-cache/csv/KNKA786.csv
LOG[INFO]: END KNKA786 (14.026153 seconds)
LOG[INFO]: START KNKA787 
LOG[INFO]: Scraped 48 rows out of 48 results
LOG[INFO]: Wrote 48 rows to search-cache/csv/KNKA787.csv
LOG[INFO]: END KNKA787 (17.395930 seconds)
LOG[INFO]: START KNKA788 
LOG[INFO]: Scraped 61 rows out of 61 results
LOG[INFO]: Wrote 61 rows to search-cache/csv/KNKA788.csv
LOG[INFO]: END

LOG[INFO]: START KNKN211 
LOG[INFO]: Scraped 84 rows out of 84 results
LOG[INFO]: Wrote 84 rows to search-cache/csv/KNKN211.csv
LOG[INFO]: END KNKN211 (15.417626 seconds)
LOG[INFO]: START KNKN212 
LOG[INFO]: Scraped 89 rows out of 89 results
LOG[INFO]: Wrote 89 rows to search-cache/csv/KNKN212.csv
LOG[INFO]: END KNKN212 (16.604815 seconds)
LOG[INFO]: START KNKN213 
LOG[INFO]: Scraped 55 rows out of 55 results
LOG[INFO]: Wrote 55 rows to search-cache/csv/KNKN213.csv
LOG[INFO]: END KNKN213 (16.124199 seconds)
LOG[INFO]: START KNKN214 
LOG[INFO]: Scraped 93 rows out of 93 results
LOG[INFO]: Wrote 93 rows to search-cache/csv/KNKN214.csv
LOG[INFO]: END KNKN214 (17.075012 seconds)
LOG[INFO]: START KNKN215 
LOG[INFO]: Scraped 66 rows out of 66 results
LOG[INFO]: Wrote 66 rows to search-cache/csv/KNKN215.csv
LOG[INFO]: END KNKN215 (15.079893 seconds)
LOG[INFO]: START KNKN216 
LOG[INFO]: Scraped 64 rows out of 64 results
LOG[INFO]: Wrote 64 rows to search-cache/csv/KNKN216.csv
LOG[INFO]: END KN

...

LOG[INFO]: START KNKN356 
LOG[INFO]: Scraped 66 rows out of 66 results
LOG[INFO]: Wrote 66 rows to search-cache/csv/KNKN356.csv
LOG[INFO]: END KNKN356 (16.676165 seconds)
LOG[INFO]: START KNKN357 
LOG[INFO]: Scraped 65 rows out of 65 results
LOG[INFO]: Wrote 65 rows to search-cache/csv/KNKN357.csv
LOG[INFO]: END KNKN357 (17.920198 seconds)
LOG[INFO]: START KNKN358 
LOG[INFO]: Scraped 90 rows out of 90 results
LOG[INFO]: Wrote 90 rows to search-cache/csv/KNKN358.csv
LOG[INFO]: END KNKN358 (16.770232 seconds)
LOG[INFO]: START KNKN359 
LOG[INFO]: Scraped 101 rows out of 101 results
LOG[INFO]: Wrote 101 rows to search-cache/csv/KNKN359.csv
LOG[INFO]: END KNKN359 (18.755313 seconds)
LOG[INFO]: START KNKN360 
LOG[INFO]: Scraped 89 rows out of 89 results
LOG[INFO]: Wrote 89 rows to search-cache/csv/KNKN360.csv
LOG[INFO]: END KNKN360 (18.129868 seconds)
LOG[INFO]: START KNKN361 
LOG[INFO]: Scraped 88 rows out of 88 results
LOG[INFO]: Wrote 88 rows to search-cache/csv/KNKN361.csv
LOG[INFO]: END

LOG[INFO]: START KNKN404 
LOG[INFO]: Scraped 101 rows out of 101 results
LOG[INFO]: Wrote 101 rows to search-cache/csv/KNKN404.csv
LOG[INFO]: END KNKN404 (18.989989 seconds)
LOG[INFO]: START KNKN405 
LOG[INFO]: Scraped 54 rows out of 54 results
LOG[INFO]: Wrote 54 rows to search-cache/csv/KNKN405.csv
LOG[INFO]: END KNKN405 (17.084981 seconds)
LOG[INFO]: START KNKN406 
LOG[INFO]: Scraped 87 rows out of 87 results
LOG[INFO]: Wrote 87 rows to search-cache/csv/KNKN406.csv
LOG[INFO]: END KNKN406 (16.544278 seconds)
LOG[INFO]: START KNKN407 
LOG[INFO]: Scraped 73 rows out of 73 results
LOG[INFO]: Wrote 73 rows to search-cache/csv/KNKN407.csv
LOG[INFO]: END KNKN407 (17.665397 seconds)
LOG[INFO]: START KNKN408 
LOG[INFO]: Scraped 82 rows out of 82 results
LOG[INFO]: Wrote 82 rows to search-cache/csv/KNKN408.csv
LOG[INFO]: END KNKN408 (17.133999 seconds)
LOG[INFO]: START KNKN409 
LOG[INFO]: Scraped 93 rows out of 93 results
LOG[INFO]: Wrote 93 rows to search-cache/csv/KNKN409.csv
LOG[INFO]: END

LOG[INFO]: START KNKN452 
LOG[INFO]: Scraped 85 rows out of 85 results
LOG[INFO]: Wrote 85 rows to search-cache/csv/KNKN452.csv
LOG[INFO]: END KNKN452 (19.085552 seconds)
LOG[INFO]: START KNKN453 
LOG[INFO]: Scraped 91 rows out of 91 results
LOG[INFO]: Wrote 91 rows to search-cache/csv/KNKN453.csv
LOG[INFO]: END KNKN453 (15.819324 seconds)
LOG[INFO]: START KNKN454 
LOG[INFO]: Scraped 51 rows out of 51 results
LOG[INFO]: Wrote 51 rows to search-cache/csv/KNKN454.csv
LOG[INFO]: END KNKN454 (17.775721 seconds)
LOG[INFO]: START KNKN455 
LOG[INFO]: Scraped 48 rows out of 48 results
LOG[INFO]: Wrote 48 rows to search-cache/csv/KNKN455.csv
LOG[INFO]: END KNKN455 (15.293595 seconds)
LOG[INFO]: START KNKN456 
LOG[INFO]: Scraped 98 rows out of 98 results
LOG[INFO]: Wrote 98 rows to search-cache/csv/KNKN456.csv
LOG[INFO]: END KNKN456 (17.578087 seconds)
LOG[INFO]: START KNKN457 
LOG[INFO]: Scraped 75 rows out of 75 results
LOG[INFO]: Wrote 75 rows to search-cache/csv/KNKN457.csv
LOG[INFO]: END KN

LOG[INFO]: START KNKN500 
LOG[INFO]: Scraped 57 rows out of 57 results
LOG[INFO]: Wrote 57 rows to search-cache/csv/KNKN500.csv
LOG[INFO]: END KNKN500 (15.588180 seconds)
LOG[INFO]: START KNKN501 
LOG[INFO]: Scraped 55 rows out of 55 results
LOG[INFO]: Wrote 55 rows to search-cache/csv/KNKN501.csv
LOG[INFO]: END KNKN501 (14.261495 seconds)
LOG[INFO]: START KNKN502 
LOG[INFO]: Scraped 44 rows out of 44 results
LOG[INFO]: Wrote 44 rows to search-cache/csv/KNKN502.csv
LOG[INFO]: END KNKN502 (15.634085 seconds)
LOG[INFO]: START KNKN503 
LOG[INFO]: Scraped 61 rows out of 61 results
LOG[INFO]: Wrote 61 rows to search-cache/csv/KNKN503.csv
LOG[INFO]: END KNKN503 (15.756977 seconds)
LOG[INFO]: START KNKN504 
LOG[INFO]: Scraped 66 rows out of 66 results
LOG[INFO]: Wrote 66 rows to search-cache/csv/KNKN504.csv
LOG[INFO]: END KNKN504 (14.683863 seconds)
LOG[INFO]: START KNKN505 
LOG[INFO]: Scraped 18 rows out of 18 results
LOG[INFO]: Wrote 18 rows to search-cache/csv/KNKN505.csv
LOG[INFO]: END KN

LOG[INFO]: START KNKN548 
LOG[INFO]: Scraped 89 rows out of 89 results
LOG[INFO]: Wrote 89 rows to search-cache/csv/KNKN548.csv
LOG[INFO]: END KNKN548 (15.552997 seconds)
LOG[INFO]: START KNKN549 
LOG[INFO]: Scraped 73 rows out of 73 results
LOG[INFO]: Wrote 73 rows to search-cache/csv/KNKN549.csv
LOG[INFO]: END KNKN549 (14.671199 seconds)
LOG[INFO]: START KNKN550 
LOG[INFO]: Scraped 72 rows out of 72 results
LOG[INFO]: Wrote 72 rows to search-cache/csv/KNKN550.csv
LOG[INFO]: END KNKN550 (16.839063 seconds)
LOG[INFO]: START KNKN551 
LOG[INFO]: Scraped 55 rows out of 55 results
LOG[INFO]: Wrote 55 rows to search-cache/csv/KNKN551.csv
LOG[INFO]: END KNKN551 (17.470300 seconds)
LOG[INFO]: START KNKN552 
LOG[INFO]: Scraped 52 rows out of 52 results
LOG[INFO]: Wrote 52 rows to search-cache/csv/KNKN552.csv
LOG[INFO]: END KNKN552 (14.677676 seconds)
LOG[INFO]: START KNKN553 
LOG[INFO]: Scraped 52 rows out of 52 results
LOG[INFO]: Wrote 52 rows to search-cache/csv/KNKN553.csv
LOG[INFO]: END KN

In [28]:
## Attempt 5

tick = time.time()
print("LOG[INFO]: START TIME is %s\n" % (time.strftime("%I:%M %p")))

tries = 0
max_tries = 5

start_index = 1000 # see attempts log below
end_index = (len(callsigns)-1)

error_locs = []
error_index = start_index

for cs in callsigns[start_index:end_index]:    
    if tries < max_tries:
        try:
            scrape_tables_for_call_sign(cs, html_cache, csv_cache, without_browser=True)
            pause = (random.randint(1,30))
            time.sleep(pause)
        except Exception as e:
            tries += 1
            error_index = callsigns.index(cs)
            error_locs.append(error_index)
            print("Error at %s, located at callsigns[%s]" % (cs, error_index))
            print("Error:", e)
            pause = (random.randint(500,800) * sqrt(tries))
            time.sleep(pause)
        finally:
            pass
    else:
        print("LOG[INFO]: Maximum number of errors (%s) reached." % (tries))
        print("Errors at:", error_locs)
        break

        
tock = time.time()
error_index = end_index  
print("LOG[INFO]: END TIME is %s (TOTAL: %s minutes)" 
      % (time.strftime("%I:%M %p"), ((tock - tick)/60))
     )
print("Completed searches from %s [%s] to %s [%s]." 
      % (callsigns[start_index], start_index, cs, error_index)
     )

error_locs

LOG[INFO]: START TIME is 02:15 PM

LOG[INFO]: START KNKN586 
LOG[INFO]: Scraped 66 rows out of 66 results
LOG[INFO]: Wrote 66 rows to search-cache/csv/KNKN586.csv
LOG[INFO]: END KNKN586 (33.104039 seconds)
LOG[INFO]: START KNKN587 
LOG[INFO]: Scraped 77 rows out of 77 results
LOG[INFO]: Wrote 77 rows to search-cache/csv/KNKN587.csv
LOG[INFO]: END KNKN587 (23.320833 seconds)
LOG[INFO]: START KNKN588 
LOG[INFO]: Scraped 58 rows out of 58 results
LOG[INFO]: Wrote 58 rows to search-cache/csv/KNKN588.csv
LOG[INFO]: END KNKN588 (21.628407 seconds)
LOG[INFO]: START KNKN589 
LOG[INFO]: Scraped 67 rows out of 67 results
LOG[INFO]: Wrote 67 rows to search-cache/csv/KNKN589.csv
LOG[INFO]: END KNKN589 (33.985138 seconds)
LOG[INFO]: START KNKN590 
LOG[INFO]: Scraped 91 rows out of 91 results
LOG[INFO]: Wrote 91 rows to search-cache/csv/KNKN590.csv
LOG[INFO]: END KNKN590 (29.607504 seconds)
LOG[INFO]: START KNKN591 
LOG[INFO]: Scraped 65 rows out of 65 results
LOG[INFO]: Wrote 65 rows to search-cach

LOG[INFO]: START KNKN634 
LOG[INFO]: Scraped 137 rows out of 137 results
LOG[INFO]: Wrote 137 rows to search-cache/csv/KNKN634.csv
LOG[INFO]: END KNKN634 (20.718136 seconds)
LOG[INFO]: START KNKN635 
LOG[INFO]: Scraped 101 rows out of 101 results
LOG[INFO]: Wrote 101 rows to search-cache/csv/KNKN635.csv
LOG[INFO]: END KNKN635 (29.105559 seconds)
LOG[INFO]: START KNKN636 
LOG[INFO]: Scraped 99 rows out of 99 results
LOG[INFO]: Wrote 99 rows to search-cache/csv/KNKN636.csv
LOG[INFO]: END KNKN636 (19.746543 seconds)
LOG[INFO]: START KNKN637 
LOG[INFO]: Scraped 62 rows out of 62 results
LOG[INFO]: Wrote 62 rows to search-cache/csv/KNKN637.csv
LOG[INFO]: END KNKN637 (18.831900 seconds)
LOG[INFO]: START KNKN638 
LOG[INFO]: Scraped 65 rows out of 65 results
LOG[INFO]: Wrote 65 rows to search-cache/csv/KNKN638.csv
LOG[INFO]: END KNKN638 (19.617341 seconds)
LOG[INFO]: START KNKN639 
LOG[INFO]: Scraped 48 rows out of 48 results
LOG[INFO]: Wrote 48 rows to search-cache/csv/KNKN639.csv
LOG[INFO]: 

LOG[INFO]: START KNKN682 
LOG[INFO]: Scraped 58 rows out of 58 results
LOG[INFO]: Wrote 58 rows to search-cache/csv/KNKN682.csv
LOG[INFO]: END KNKN682 (16.887529 seconds)
LOG[INFO]: START KNKN683 
LOG[INFO]: Scraped 39 rows out of 39 results
LOG[INFO]: Wrote 39 rows to search-cache/csv/KNKN683.csv
LOG[INFO]: END KNKN683 (16.016110 seconds)
LOG[INFO]: START KNKN684 
LOG[INFO]: Scraped 66 rows out of 66 results
LOG[INFO]: Wrote 66 rows to search-cache/csv/KNKN684.csv
LOG[INFO]: END KNKN684 (15.919924 seconds)
LOG[INFO]: START KNKN685 
LOG[INFO]: Scraped 56 rows out of 56 results
LOG[INFO]: Wrote 56 rows to search-cache/csv/KNKN685.csv
LOG[INFO]: END KNKN685 (18.753252 seconds)
LOG[INFO]: START KNKN686 
LOG[INFO]: Scraped 44 rows out of 44 results
LOG[INFO]: Wrote 44 rows to search-cache/csv/KNKN686.csv
LOG[INFO]: END KNKN686 (15.266851 seconds)
LOG[INFO]: START KNKN687 
LOG[INFO]: Scraped 37 rows out of 37 results
LOG[INFO]: Wrote 37 rows to search-cache/csv/KNKN687.csv
LOG[INFO]: END KN

LOG[INFO]: START KNKN730 
LOG[INFO]: Scraped 99 rows out of 99 results
LOG[INFO]: Wrote 99 rows to search-cache/csv/KNKN730.csv
LOG[INFO]: END KNKN730 (17.284178 seconds)
LOG[INFO]: START KNKN731 
LOG[INFO]: Scraped 39 rows out of 39 results
LOG[INFO]: Wrote 39 rows to search-cache/csv/KNKN731.csv
LOG[INFO]: END KNKN731 (15.460748 seconds)
LOG[INFO]: START KNKN732 
LOG[INFO]: Scraped 34 rows out of 34 results
LOG[INFO]: Wrote 34 rows to search-cache/csv/KNKN732.csv
LOG[INFO]: END KNKN732 (14.816221 seconds)
LOG[INFO]: START KNKN733 
LOG[INFO]: Scraped 34 rows out of 34 results
LOG[INFO]: Wrote 34 rows to search-cache/csv/KNKN733.csv
LOG[INFO]: END KNKN733 (15.304766 seconds)
LOG[INFO]: START KNKN734 
LOG[INFO]: Scraped 35 rows out of 35 results
LOG[INFO]: Wrote 35 rows to search-cache/csv/KNKN734.csv
LOG[INFO]: END KNKN734 (15.576643 seconds)
LOG[INFO]: START KNKN735 
LOG[INFO]: Scraped 51 rows out of 51 results
LOG[INFO]: Wrote 51 rows to search-cache/csv/KNKN735.csv
LOG[INFO]: END KN

LOG[INFO]: START KNKN778 
LOG[INFO]: Scraped 41 rows out of 41 results
LOG[INFO]: Wrote 41 rows to search-cache/csv/KNKN778.csv
LOG[INFO]: END KNKN778 (24.650686 seconds)
LOG[INFO]: START KNKN779 
LOG[INFO]: Scraped 52 rows out of 52 results
LOG[INFO]: Wrote 52 rows to search-cache/csv/KNKN779.csv
LOG[INFO]: END KNKN779 (18.011184 seconds)
LOG[INFO]: START KNKN780 
LOG[INFO]: Scraped 35 rows out of 35 results
LOG[INFO]: Wrote 35 rows to search-cache/csv/KNKN780.csv
LOG[INFO]: END KNKN780 (17.465625 seconds)
LOG[INFO]: START KNKN781 
LOG[INFO]: Scraped 71 rows out of 71 results
LOG[INFO]: Wrote 71 rows to search-cache/csv/KNKN781.csv
LOG[INFO]: END KNKN781 (19.185105 seconds)
LOG[INFO]: START KNKN782 
LOG[INFO]: Scraped 84 rows out of 84 results
LOG[INFO]: Wrote 84 rows to search-cache/csv/KNKN782.csv
LOG[INFO]: END KNKN782 (19.266888 seconds)
LOG[INFO]: START KNKN783 
LOG[INFO]: Scraped 52 rows out of 52 results
LOG[INFO]: Wrote 52 rows to search-cache/csv/KNKN783.csv
LOG[INFO]: END KN

...

LOG[INFO]: START WPQS659 
Error at WPQS659, located at callsigns[2442]
Error: list index out of range
LOG[INFO]: START WPQS770 
LOG[INFO]: Scraped 1 rows out of 1 results
LOG[INFO]: Wrote 1 rows to search-cache/csv/WPQS770.csv
LOG[INFO]: END WPQS770 (16.608152 seconds)
LOG[INFO]: START WPQT310 
LOG[INFO]: Scraped 2 rows out of 2 results
LOG[INFO]: Wrote 2 rows to search-cache/csv/WPQT310.csv
LOG[INFO]: END WPQT310 (17.691647 seconds)
LOG[INFO]: START WPQT480 
LOG[INFO]: Scraped 1 rows out of 1 results
LOG[INFO]: Wrote 1 rows to search-cache/csv/WPQT480.csv
LOG[INFO]: END WPQT480 (15.287515 seconds)
LOG[INFO]: START WPQT544 
LOG[INFO]: Scraped 8 rows out of 8 results
LOG[INFO]: Wrote 8 rows to search-cache/csv/WPQT544.csv
LOG[INFO]: END WPQT544 (17.300470 seconds)
LOG[INFO]: START WPQX394 
LOG[INFO]: Scraped 2 rows out of 2 results
LOG[INFO]: Wrote 2 rows to search-cache/csv/WPQX394.csv
LOG[INFO]: END WPQX394 (14.424394 seconds)
LOG[INFO]: START WPQX513 
LOG[INFO]: Scraped 2 rows out of

LOG[INFO]: START WPUH619 
LOG[INFO]: Scraped 24 rows out of 24 results
LOG[INFO]: Wrote 24 rows to search-cache/csv/WPUH619.csv
LOG[INFO]: END WPUH619 (18.002129 seconds)
LOG[INFO]: START WPUH800 
LOG[INFO]: Scraped 9 rows out of 9 results
LOG[INFO]: Wrote 9 rows to search-cache/csv/WPUH800.csv
LOG[INFO]: END WPUH800 (15.775624 seconds)
LOG[INFO]: START WPUH805 
LOG[INFO]: Scraped 20 rows out of 20 results
LOG[INFO]: Wrote 20 rows to search-cache/csv/WPUH805.csv
LOG[INFO]: END WPUH805 (15.344882 seconds)
LOG[INFO]: START WPUJ480 
LOG[INFO]: Scraped 21 rows out of 21 results
LOG[INFO]: Wrote 21 rows to search-cache/csv/WPUJ480.csv
LOG[INFO]: END WPUJ480 (17.033148 seconds)
LOG[INFO]: START WPUK842 
LOG[INFO]: Scraped 24 rows out of 24 results
LOG[INFO]: Wrote 24 rows to search-cache/csv/WPUK842.csv
LOG[INFO]: END WPUK842 (16.185658 seconds)
LOG[INFO]: START WPUN203 
LOG[INFO]: Scraped 3 rows out of 3 results
LOG[INFO]: Wrote 3 rows to search-cache/csv/WPUN203.csv
LOG[INFO]: END WPUN203 

LOG[INFO]: START WQIF921 
LOG[INFO]: Scraped 8 rows out of 8 results
LOG[INFO]: Wrote 8 rows to search-cache/csv/WQIF921.csv
LOG[INFO]: END WQIF921 (41.069039 seconds)
LOG[INFO]: START WQIK801 
LOG[INFO]: Scraped 7 rows out of 7 results
LOG[INFO]: Wrote 7 rows to search-cache/csv/WQIK801.csv
LOG[INFO]: END WQIK801 (15.123960 seconds)
LOG[INFO]: START WQIQ306 
LOG[INFO]: Scraped 24 rows out of 24 results
LOG[INFO]: Wrote 24 rows to search-cache/csv/WQIQ306.csv
LOG[INFO]: END WQIQ306 (17.214280 seconds)
LOG[INFO]: START WQIQ308 
LOG[INFO]: Scraped 23 rows out of 23 results
LOG[INFO]: Wrote 23 rows to search-cache/csv/WQIQ308.csv
LOG[INFO]: END WQIQ308 (17.160140 seconds)
LOG[INFO]: START WQIQ784 
LOG[INFO]: Scraped 16 rows out of 16 results
LOG[INFO]: Wrote 16 rows to search-cache/csv/WQIQ784.csv
LOG[INFO]: END WQIQ784 (15.585452 seconds)
LOG[INFO]: START WQJE415 
LOG[INFO]: Scraped 20 rows out of 20 results
LOG[INFO]: Wrote 20 rows to search-cache/csv/WQJE415.csv
LOG[INFO]: END WQJE415 

LOG[INFO]: START WQVD597 
LOG[INFO]: Scraped 2 rows out of 2 results
LOG[INFO]: Wrote 2 rows to search-cache/csv/WQVD597.csv
LOG[INFO]: END WQVD597 (19.277631 seconds)
LOG[INFO]: START WQVS689 
LOG[INFO]: Scraped 1 rows out of 1 results
LOG[INFO]: Wrote 1 rows to search-cache/csv/WQVS689.csv
LOG[INFO]: END WQVS689 (15.553544 seconds)
LOG[INFO]: START WQVX228 
LOG[INFO]: Scraped 2 rows out of 2 results
LOG[INFO]: Wrote 2 rows to search-cache/csv/WQVX228.csv
LOG[INFO]: END WQVX228 (17.561465 seconds)
LOG[INFO]: START WQVX230 
LOG[INFO]: Scraped 2 rows out of 2 results
LOG[INFO]: Wrote 2 rows to search-cache/csv/WQVX230.csv
LOG[INFO]: END WQVX230 (16.598442 seconds)
LOG[INFO]: START WQVZ380 
LOG[INFO]: Scraped 2 rows out of 2 results
LOG[INFO]: Wrote 2 rows to search-cache/csv/WQVZ380.csv
LOG[INFO]: END WQVZ380 (15.936846 seconds)
LOG[INFO]: START WQWD617 
LOG[INFO]: Scraped 9 rows out of 9 results
LOG[INFO]: Wrote 9 rows to search-cache/csv/WQWD617.csv
LOG[INFO]: END WQWD617 (16.701883 s

[1470, 2022, 2300, 2442]